# Classifier V2

The Goal: Use the brightness levels of pixels in the image dataset to classify types of fashion items.

Useful links:   
https://towardsdatascience.com/the-4-convolutional-neural-network-models-that-can-classify-your-fashion-images-9fe7f3e5399d
https://github.com/khanhnamle1994/fashion-mnist/blob/master/CNN-1Conv.ipynb

## 1. Load and Split Dataset

Import required packages

In [2]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Import Keras libraries
from tensorflow.python import keras

In [3]:
# Load training and test data into dataframes
data_train = pd.read_csv('data/fashion-mnist_train.csv')
data_test = pd.read_csv('data/fashion-mnist_test.csv')

# X forms the training images, and y forms the training labels
X = np.array(data_train.iloc[:, 1:])
y = to_categorical(np.array(data_train.iloc[:, 0]))

# split original training data to sub-training (80%) and validation data (20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

# X_test forms the test images, and y_test forms the test labels
X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

## 2. Normalise the Datasets

In [4]:
# Each image's dimension is 28 x 28
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# Prepare the training images
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_train /= 255

# Prepare the test images
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_test = X_test.astype('float32')
X_test /= 255

# Prepare the validation images
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_val = X_val.astype('float32')
X_val /= 255

## 3. CNN with 1 Convolutional Layer

CNN takes as input tensors of shape (image_height, image_width, image_channels) where image channels ware 1 if pixel brightness is given, or would be 3 if rgb of each pixel were given. In this case, configure the CNN to process inputs of size (28, 28, 1), the FashionMNIST images. Do this by passing the argument input_shape=(28, 28, 1) to the first layer.
* The 1st layer is a Conv2D layer for the convolution operation that extracts features from the input images by sliding a convolution filter over the input to produce a feature map. Here choose feature map with size 5 x 5.
* The 2nd layer is a MaxPooling2D layer for the max-pooling operation that reduces the dimensionality of each feature, which helps shorten training time and reduce number of parameters. Here choose the pooling window with size 2 x 2.
* To combat overfititng, add a Dropout layer as the 3rd layer, a powerful regularization technique. Dropout is the method used to reduce overfitting, whcih forces the model to learn multiple independent representations of the same data by randomly disabling neurons in the learning phase. In this model, dropout will randomnly disable 20% of the neurons.
* The next step is to feed the last output tensor into a stack of Dense layers, otherwise known as fully-connected layers. These densely connected classifiers process vectors, which are 1D, whereas the current output is a 3D tensor. Thus, need to flatten the 3D outputs to 1D, and then add 2 Dense layers on top.
* Do a 10-way classification (as there are 10 classes of fashion images), using a final layer with 10 outputs and a softmax activation. Softmax activation enables calculation of the output based on probabilities. Each class is assigned a probability and the class with the maximum probability is the model’s output for the input.

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.2))

cnn1.add(Flatten())

cnn1.add(Dense(128, activation='relu'))
cnn1.add(Dense(10, activation='softmax'))